In [19]:
import numpy as np
import pandas as pd
import config
import argparse
import os
import shutil

from math import sqrt
from util import *
from md2nmr import *



In [33]:
def main(batch, topo=None, traj=None):
    print("###############    Software starting now.    ###############")
    print("Input arguments and config currently using:")
    print(f"batch: {batch}")
    if not batch:
        print(f"topo: {topo}")
        print(f"traj: {traj}")

    for key, value in config.__dict__.items():
        if not key.startswith("__"):
            print(f"{key}: {value}")

    print("###############    Calculation starting...    ###############")
    
    if batch:
        print('Calculation using batch style.')
        print('Following files will be used in calculation:')
        print([os.path.join(wd, prefix + '.pdb') for prefix in config.prefix_list])
        
        for prefix in config.prefix_list:
            print(f'Generating NMR relaxation for: {prefix}')
            FTOPN = os.path.join(wd, prefix + '.pdb')
            FMDN = os.path.join(wd, prefix + '.xtc')
            gen_nmr_relaxation(FTOPN, FMDN, prefix)
    else:
        print('Calculation using single traj style.')
        FTOPN = os.path.join(wd, str(topo))
        FMDN = os.path.join(wd, str(traj))
        prefix = traj[:-4] #define the prefix for output files.
        gen_nmr_relaxation(FTOPN, FMDN, prefix)

    print("###############    Calculation completed. Software exiting...    ###############")

In [35]:
batch_mode = False
topo_file = 'vanilla_run1.pdb'
traj_file = 'vanilla_run1.xtc'

In [37]:
main(batch=batch_mode, topo=topo_file, traj=traj_file)

###############    Software starting now.    ###############
Input arguments and config currently using:
batch: False
topo: vanilla_run1.pdb
traj: vanilla_run1.xtc
n_split: 20
tau_max: 1.8
B0: 14.1
resid: [[0, 10]]
use_chain_ids: False
chain_ids: ['C', 'G']
wd: ../data/
od: ../data/
dt: 100
n_exp: 3
use_align: True
tumbling_time: None
prefix_list: ['vanilla_run1']
###############    Calculation starting...    ###############
Calculation using single traj style.
Loading trajectory...
Trajectory loaded.
Aligning trajectory...
Alignment completed.
Total frames of MD trajectory loaded:  22406
vec shape after split: (20, 1120, 11, 3)
###############    Calculation completed. Software exiting...    ###############
